<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Fine_tuning_BERT__multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
!pip install -q transformers datasets

     |████████████████████████████████| 4.4 MB 30.9 MB/s 
     |████████████████████████████████| 362 kB 55.0 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 1.1 MB 49.9 MB/s 
     |████████████████████████████████| 140 kB 74.9 MB/s 
     |████████████████████████████████| 212 kB 55.1 MB/s 
     |████████████████████████████████| 127 kB 57.0 MB/s 
     |████████████████████████████████| 144 kB 25.2 MB/s 
     |████████████████████████████████| 271 kB 63.3 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/drive/MyDrive/PaperA3/Final_Dataset/df_raw_TFIDF_L2_G02.csv')
df = df.drop('Unnamed: 0', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [5]:
df.head()

,HADM_ID,comment_text,L_1.0,L_2.0,L_3.0,L_4.0,L_5.0,L_6.0,L_7.0,L_8.0,L_9.0,L_10.0,L_11.0,L_12.0,L_13.0,L_14.0,L_15.0,L_16.0,L_17.0,L_18.0
0,157267.0,"Mucous body substance, Twice a day, clarithrom...",False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,123103.0,"Machine, Done (qualifier value), Bromdimethoxy...",False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
2,139931.0,"Structure of right renal artery, Chemical inju...",False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,198295.0,"Calculus of gallbladder with cholecystitis, He...",False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
4,177962.0,"Aplastic, Three Vessel Coronary Disease, Scheu...",False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df

In [6]:
from sklearn.model_selection import train_test_split
train_size=0.8
train, rem= train_test_split(df, train_size=0.8)
test_size = 0.5
valid, test = train_test_split(rem, test_size=0.5)

print(train.shape)
print(valid.shape)
print(test.shape)



(21072, 20)
(2634, 20)
(2635, 20)


In [7]:
from datasets import Dataset
import datasets
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
valid_dataset = Dataset.from_pandas(valid)
dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset,"valid":valid_dataset})

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['HADM_ID', 'comment_text', 'L_1.0', 'L_2.0', 'L_3.0', 'L_4.0', 'L_5.0', 'L_6.0', 'L_7.0', 'L_8.0', 'L_9.0', 'L_10.0', 'L_11.0', 'L_12.0', 'L_13.0', 'L_14.0', 'L_15.0', 'L_16.0', 'L_17.0', 'L_18.0', '__index_level_0__'],
        num_rows: 21072
    })
    test: Dataset({
        features: ['HADM_ID', 'comment_text', 'L_1.0', 'L_2.0', 'L_3.0', 'L_4.0', 'L_5.0', 'L_6.0', 'L_7.0', 'L_8.0', 'L_9.0', 'L_10.0', 'L_11.0', 'L_12.0', 'L_13.0', 'L_14.0', 'L_15.0', 'L_16.0', 'L_17.0', 'L_18.0', '__index_level_0__'],
        num_rows: 2635
    })
    valid: Dataset({
        features: ['HADM_ID', 'comment_text', 'L_1.0', 'L_2.0', 'L_3.0', 'L_4.0', 'L_5.0', 'L_6.0', 'L_7.0', 'L_8.0', 'L_9.0', 'L_10.0', 'L_11.0', 'L_12.0', 'L_13.0', 'L_14.0', 'L_15.0', 'L_16.0', 'L_17.0', 'L_18.0', '__index_level_0__'],
        num_rows: 2634
    })
})

Let's check the first example of the training split:

In [9]:
example = dataset['train'][0]
example

{'HADM_ID': 105981.0,
 'L_1.0': False,
 'L_10.0': False,
 'L_11.0': False,
 'L_12.0': False,
 'L_13.0': False,
 'L_14.0': False,
 'L_15.0': False,
 'L_16.0': True,
 'L_17.0': False,
 'L_18.0': True,
 'L_2.0': False,
 'L_3.0': False,
 'L_4.0': False,
 'L_5.0': False,
 'L_6.0': False,
 'L_7.0': False,
 'L_8.0': False,
 'L_9.0': False,
 '__index_level_0__': 16079,
 'comment_text': 'gamma hydroxybutyrate, Snoring, succinylcholine, etomidate, ABLEPHARON-MACROSTOMIA SYNDROME, propofol, GEN1 gene, Partial pressure of Oxygen, Wounds and Injuries, Myelofibrosis, Vomiting, Refractory anemias, Pain, Pneumothorax, Pleural effusion disorder, Edema, '}

In [10]:
example = dataset['train'][0]
example

{'HADM_ID': 105981.0,
 'L_1.0': False,
 'L_10.0': False,
 'L_11.0': False,
 'L_12.0': False,
 'L_13.0': False,
 'L_14.0': False,
 'L_15.0': False,
 'L_16.0': True,
 'L_17.0': False,
 'L_18.0': True,
 'L_2.0': False,
 'L_3.0': False,
 'L_4.0': False,
 'L_5.0': False,
 'L_6.0': False,
 'L_7.0': False,
 'L_8.0': False,
 'L_9.0': False,
 '__index_level_0__': 16079,
 'comment_text': 'gamma hydroxybutyrate, Snoring, succinylcholine, etomidate, ABLEPHARON-MACROSTOMIA SYNDROME, propofol, GEN1 gene, Partial pressure of Oxygen, Wounds and Injuries, Myelofibrosis, Vomiting, Refractory anemias, Pain, Pneumothorax, Pleural effusion disorder, Edema, '}

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [12]:
labels = [label for label in dataset['train'].features.keys() if label not in ['HADM_ID', 'comment_text','__index_level_0__']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['L_1.0',
 'L_2.0',
 'L_3.0',
 'L_4.0',
 'L_5.0',
 'L_6.0',
 'L_7.0',
 'L_8.0',
 'L_9.0',
 'L_10.0',
 'L_11.0',
 'L_12.0',
 'L_13.0',
 'L_14.0',
 'L_15.0',
 'L_16.0',
 'L_17.0',
 'L_18.0']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [13]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["comment_text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f5e6c673200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [15]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [16]:
tokenizer.decode(example['input_ids'])

'[CLS] gamma hydroxybutyrate, snoring, succinylcholine, etomidate, ablepharon - macrostomia syndrome, propofol, gen1 gene, partial pressure of oxygen, wounds and injuries, myelofibrosis, vomiting, refractory anemias, pain, pneumothorax, pleural effusion disorder, edema, [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [17]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0]

In [18]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['L_16.0', 'L_18.0']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [19]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [21]:
batch_size = 32
metric_name = "f1"

In [22]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [23]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [24]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [25]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 13091, 18479, 18037,  8569,  3723, 11657,  1010,  1055, 12131,
         2075,  1010, 10514, 14693,  4890, 29358, 18861,  2063,  1010,  3802,
        20936, 13701,  1010,  2583, 21890,  4948,  1011, 26632, 16033, 10092,
         8715,  1010, 17678, 11253,  4747,  1010,  8991,  2487,  4962,  1010,
         7704,  3778,  1997,  7722,  1010,  8710,  1998,  6441,  1010,  2026,
        18349,  8873, 12618,  6190,  1010, 24780,  1010, 25416, 22648,  7062,
         2019, 17577,  2015,  1010,  3255,  1010,  1052,  2638,  2819, 29288,
         2527,  2595,  1010, 20228, 11236,  2389,  1041,  4246, 14499,  8761,
         1010,  3968, 14545,  1010,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [26]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput([('loss',
                           tensor(0.7159, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[ 0.0914, -0.5682, -0.0234,  0.2828,  0.0230, -0.2990,  0.1349, -0.1551,
                                     0.4758,  0.7144,  0.1318, -0.1784,  0.2168,  0.0569,  0.2235,  0.4921,
                                    -0.0367,  0.2036]], grad_fn=<AddmmBackward0>))])

Let's start training!

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21072
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3295


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.314300,0.256934,0.442779,0.649328,0.105543
2,0.256200,0.244109,0.498975,0.678189,0.129461
3,0.244900,0.238683,0.532610,0.698592,0.141610
4,0.227100,0.237632,0.528339,0.695881,0.142749
5,0.221400,0.236807,0.534312,0.700097,0.148443


***** Running Evaluation *****
  Num examples = 2634
  Batch size = 32
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-659
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-659/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-659/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-659/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-659/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2634
  Batch size = 32
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-1318
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-1318/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-1318/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-1318/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/check

TrainOutput(global_step=3295, training_loss=0.24731824814097472, metrics={'train_runtime': 2415.6432, 'train_samples_per_second': 43.616, 'train_steps_per_second': 1.364, 'total_flos': 6931340794552320.0, 'train_loss': 0.24731824814097472, 'epoch': 5.0})

In [42]:
trainer.save_model('/content/drive/MyDrive/PaperA3/Results')

Saving model checkpoint to /content/drive/MyDrive/PaperA3/Results
Configuration saved in /content/drive/MyDrive/PaperA3/Results/config.json
Model weights saved in /content/drive/MyDrive/PaperA3/Results/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/PaperA3/Results/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/PaperA3/Results/special_tokens_map.json


## Evaluate

After training, we evaluate our model on the validation set.

In [29]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2634
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.1484434320425209,
 'eval_f1': 0.5343116098403108,
 'eval_loss': 0.23680689930915833,
 'eval_roc_auc': 0.7000965379933259,
 'eval_runtime': 19.6904,
 'eval_samples_per_second': 133.771,
 'eval_steps_per_second': 4.215}

## Inference

Let's test the model on a new sentence:

In [33]:
text = "gamma hydroxybutyrate, snoring, succinylcholine, etomidate, ablepharon - macrostomia syndrome, propofol, gen1 gene, partial pressure of oxygen, wounds and injuries, myelofibrosis, vomiting, refractory anemias, pain, pneumothorax, pleural effusion disorder, edema"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [34]:
logits = outputs.logits
logits.shape

torch.Size([1, 18])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [41]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['L_16.0']
